In [2]:
!pip install pyopenpose

ERROR: Could not find a version that satisfies the requirement pyopenpose (from versions: none)
ERROR: No matching distribution found for pyopenpose


In [1]:
import cv2
import numpy as np
import pyopenpose as op
import time
import sys
import argparse

# Check if running in Jupyter
in_jupyter = 'ipykernel' in sys.modules

if not in_jupyter:
    parser = argparse.ArgumentParser()
    parser.add_argument('--input', type=str, default='0', help='Input video file or webcam index')
    parser.add_argument('--output', type=str, help='Output video file (optional)')
    parser.add_argument('--model_folder', type=str, required=True, help='Path to OpenPose models')
    args = parser.parse_args()
else:
    # Default values for Jupyter
    class Args:
        def __init__(self):
            self.input = '0'
            self.output = None
            self.model_folder = "<path_to_openpose_models>"
    args = Args()

# Custom OpenPose params
params = {
    "model_folder": args.model_folder,
    "net_resolution": "-1x368"
}

# Starting OpenPose
opWrapper = op.WrapperPython()
opWrapper.configure(params)
opWrapper.start()

# Open video capture
cap = cv2.VideoCapture(args.input if not args.input.isdigit() else int(args.input))
if not cap.isOpened():
    print("Error: Could not open video source.")
    sys.exit(1)

# Get video properties for potential output
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Initialize video writer if output is specified
out = None
if args.output:
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(args.output, fourcc, 20.0, (frame_width, frame_height))

# Lists to store performance metrics and keypoints
fps_list = []
all_keypoints = []

while cap.isOpened():
    start_time = time.time()
    
    ret, frame = cap.read()
    if not ret:
        print("Error: Can't receive frame (stream end?). Exiting ...")
        break
    
    # Create a new datum
    datum = op.Datum()
    datum.cvInputData = frame
    
    # Process the frame
    opWrapper.emplaceAndPop(op.VectorDatum([datum]))
    
    # Extract keypoints
    try:
        keypoints = datum.poseKeypoints
        if keypoints is not None:
            all_keypoints.append(keypoints)
    except:
        pass
    
    # Get the output image with keypoints drawn
    image = datum.cvOutputData
    
    # Calculate and display FPS
    end_time = time.time()
    fps = 1 / (end_time - start_time)
    fps_list.append(fps)
    cv2.putText(image, f'FPS: {fps:.2f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Display the image
    cv2.imshow('OpenPose Detection', image)
    
    # Write frame to output video if specified
    if out:
        out.write(image)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
if out:
    out.release()
cv2.destroyAllWindows()

# Print performance metrics
average_fps = np.mean(fps_list)
print(f"Average FPS: {average_fps:.2f}")
print(f"Total frames processed: {len(fps_list)}")
print(f"Total keypoint sets collected: {len(all_keypoints)}")

ModuleNotFoundError: No module named 'pyopenpose'

In [ ]:
!pip install pyopenpose